In [28]:

import base64
import io
import math
import operator
import random
import time
import urllib


In [29]:
import matplotlib.pyplot as plt


In [30]:
from PIL import Image


In [31]:
import rtree

In [32]:
import shapefile


In [33]:
from shapely.geometry import Polygon, Point


In [34]:
import pyproj


In [35]:
def slice_at(plist, indices):
    '''
    Slices the list ll at the indices indicated.
    '''
    indices.append(len(plist))
    for fro, to in zip(indices, indices[1:]):
        yield plist[fro:to]

In [36]:

def plot_country(name, parts, center, radius):
    fig, ax = plt.subplots(figsize=(1.8, 1.8))

    circle = plt.Circle(center, radius, color='k', fill=False)
    for part in parts:
        #color = random.choice('bgrcmk')
        color = 'k'
        for point1, point2 in zip(part, part[1:]):
            ax.plot(
                [point1[0], point2[0]],
                [point1[1], point2[1]],
                color+'-')
    ax.add_artist(circle)
    ax.set_aspect('equal', 'datalim')
    imgdata = io.BytesIO()
    #ax.set_title(name)
    plt.axis('off')
    #plt.savefig(name.lower().replace(' ', '-') + '.png', transparent=True)
    #plt.show()
    plt.savefig('tmp.png', format='png', transparent=True)
    img = Image.open('tmp.png')
    png_info = img.info
    img.convert('P').save(imgdata, 'PNG', **png_info)
    imgdata.seek(0)
    plt.close(fig)
    return imgdata

In [37]:
def neighbours(center, radius, delta):
    cmov = delta
    rmov = delta
    cms = [cmov*20, cmov, 0, -cmov, -cmov*20]
    rms = [rmov*20, rmov, 0, -rmov, -rmov*20]
    for cx_mov in cms:
        for cy_mov in cms:
            for r_mov in rms:
                if radius + r_mov > 0:
                    yield ((center[0] + cx_mov, center[1] + cy_mov),
                           radius + r_mov)

In [38]:
def maxmin(arr):
    _max = arr[0]
    _min = arr[0]
    for x in arr:
        if x > _max:
            _max = x
        elif x < _min:
            _min = x
    return _max, _min

In [39]:

def heuristic(polygons, index, c, r, delta, iters=200):
    best_coeff = 0
    parts_area = sum(map(lambda x: x.area, polygons))
    is_ = 0
    for _ in range(iters):
        is_ += 1
        mejoro = False
        for c_, r_ in neighbours(c, r, delta):
            circle = Point(c_).buffer(r_)
            indices = range(len(polygons))
            if circle.bounds != ():
                indices = [int(i) for i in index.intersection(circle.bounds)]
            else:
                print('WARNING: circle.bounds == (). '\
                      'center = %s, radius = %.3f' % (str(c_), r_))
            intersection_area = sum(
                [polygons[i].intersection(circle).area for i in indices])
            coeff = intersection_area / max(math.pi * r_ * r_, parts_area)
            #print(c_, r_)
            #print(intersection_area, circle_area, parts_area)
            if coeff > best_coeff:
                mejoro = True
                c = c_
                r = r_
                best_coeff = coeff
        if not mejoro:
            break
    return c, r, best_coeff, is_

In [40]:
def circle_inside(xmin, ymin, xmax, ymax):
    center = [(xmin + xmax) / 2, (ymin + ymax) / 2]
    radius = min((xmax - xmin) / 2, (ymax - ymin) / 2)
    return center, radius

In [41]:
def circle_outside(xmin, ymin, xmax, ymax):
    center = [(xmin + xmax) / 2, (ymin + ymax) / 2]
    radius = math.sqrt((xmax - xmin)**2.0 + (ymax - ymin)**2.0) / 2
    return center, radius

In [42]:

def analyze_both_circles(polys, index, min_x, min_y, max_x, max_y):
    cinside, rinside = circle_inside(min_x, min_y, max_x, max_y)
    coutside, routside = circle_outside(min_x, min_y, max_x, max_y)
    c_i, r_i, best_i, iin = heuristic(
        polys, index, cinside, rinside, rinside / 200)
    print('[%d, ' % iin, end='', flush=True)
    c_o, r_o, best_o, iout = heuristic(
        polys, index, coutside, routside, routside / 200)
    print('%d]' % iout)
    best_coeff_ = max(best_i, best_o)
    center_ = c_i if best_i > best_o else c_o
    radius_ = r_i if best_i > best_o else r_o
    return center_, radius_, best_coeff_

In [43]:

def find_best_circle(parts, minx, miny, maxx, maxy):
    polys = [Polygon(part).buffer(0) for part in parts]
    index = rtree.index.Index()
    for i, poly in enumerate(polys):
        index.insert(i, poly.bounds)
    print('Analyzing best overall circle...  Iterations: ', end='', flush=True)
    c, r, best_coeff = analyze_both_circles(
        polys, index, minx, miny, maxx, maxy)
    best_polygons = []
    if len(polys) > 1:
        polygons_area = sorted(
            [(poly, poly.area) for poly in polys],
            key=lambda x: x[1], reverse=True)
        areas = list(map(lambda r: r[1], polygons_area))
        if areas[0] > sum(areas[1:]):
            best_polygons = [polygons_area[0][0]]
        else:
            best_polygons = list(map(lambda x: x[0], polygons_area))[:5]
    i = 0
    for part in best_polygons:
        if len(polys) == 1:
            break
        i += 1
        print('Analyzing best part circle... (%d out of %d) Iterations: ' \
            % (i, len(polys)), end='', flush=True)
        x, y = part.exterior.coords.xy
        max_x, min_x = maxmin(x)
        max_y, min_y = maxmin(y)
        center_, radius_, best_coeff_ = analyze_both_circles(
            polys, index, min_x, min_y, max_x, max_y)
        if best_coeff_ > best_coeff:
            c = center_
            r = radius_
            best_coeff = best_coeff_
    # fine tune it.
    print('Fine-tuning the circle... Iterations: ', end='', flush=True)
    c, r, best_coeff, iters = heuristic(polys, index, c, r, r / 1000, 1000)
    print('[%d]' % iters)
    return c, r, best_coeff


In [44]:
def tuple2list(t):
    t = tuple(t)[0]
    return [t[0], t[1]]

In [45]:
def concat(ls):
    return [j for i in ls for j in i]

In [46]:

def mean(ls):
    return float(sum(ls)) / len(ls)

In [47]:
INPUT_FILE = 'LA'
FROM_PROJ = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'


In [48]:
def main():
    countries = shapefile.Reader(INPUT_FILE).shapeRecords()
    results = {}

In [49]:
fields =shapefile.Reader(INPUT_FILE).fields[1:] 
field_names = [field[0] for field in fields] 
# construction of a dctionary field_name:value  
for r in shapefile.Reader(INPUT_FILE).shapeRecords():  
     atr = dict(zip(field_names, r.record))  


In [50]:
print(field_names)

['objectid', 'lad17cd', 'lad17nm', 'lad17nmw', 'bng_e', 'bng_n', 'long', 'lat', 'st_areasha', 'st_lengths']


In [51]:
country = field_names[2]

In [52]:
print(country)

lad17nm


In [ ]:

def main():
    countries = shapefile.Reader(INPUT_FILE).shapeRecords()
    results = {}

    for country in countries:
        t0 = time.process_time()
        name = country.record[2]
        # if name != 'Nauru': continue
        print('==== %s (%d -> %d) ====' % (
            name, len(country.shape.parts), len(country.shape.points)))
        points = list(map(lambda x: x, country.shape.points))

        parts_ = list(country.shape.parts) + [len(country.shape.points)]
        largest_part = (0, 1)
        for x1, x2 in zip(parts_, parts_[1:]):
            if x2 - x1 > largest_part[1] - largest_part[0]:
                largest_part = (x1, x2)
        print(parts_, largest_part)

        # Get the random points from the largest part.
        random_points = list(
            map(
                list,
                random.sample(points[largest_part[0]:largest_part[1]], 2)))

        # Get the middle points between them.
        lon, lat = list(
            pyproj.Geod(ellps='WGS84').npts(
                random_points[0][0], random_points[0][1],
                random_points[1][0], random_points[1][1],
                1)  # Just 1 point in-between them.
            )[0]

#         to_proj = '+proj=aeqd +R=6371000 +lat_0=%.2f +lon_0=%.2f ' \
#                   '+no_defs' % (lat, lon)
#         print('Projection: %s' % to_proj)

#         transform = lambda g: pyproj.transform(
#             pyproj.Proj(FROM_PROJ),
#             pyproj.Proj(to_proj),
#             g[0], g[1])
#         points = list(map(transform, points))

        parts = list(slice_at(points, country.shape.parts))
        max_xs, min_xs = maxmin(list(map(lambda t: t[0], points)))
        max_ys, min_ys = maxmin(list(map(lambda t: t[1], points)))

        center, radius, best_coeff = find_best_circle(
            parts, min_xs, min_ys, max_xs, max_ys)

        print('Result info: %f (%.4f, %.4f) %.4f' \
            % (best_coeff, center[0], center[1], radius))
        uri = 'data:image/png;base64,' + urllib.parse.quote(
            base64.b64encode(
                plot_country(name, parts, center, radius).getvalue()))
        print('Finished plotting %s.' % name)
        results[name] = (best_coeff, '<img src="%s" />' % uri)

    f = open('table', 'w')
    # Does python use theorems for free?
    # (https://www.mpi-sws.org/~dreyer/tor/papers/wadler.pdf)
    f.write(
        '\n'.join(
            map(lambda s: ' | '.join(map(str, s)),
                map(lambda r: [r[0]+1, r[1][0],
                               '%.3f' % r[1][1][0],
                               r[1][1][1]],
                    enumerate(
                        sorted(results.items(),
                               key=lambda z: (operator.itemgetter(1)(z))[0],
                               reverse=True))))))8
    f.close()

In [ ]:
main()

==== Hartlepool (2 -> 394) ====
[0, 390, 394] (0, 390)
Analyzing best overall circle...  Iterations: [15, 17]
Analyzing best part circle... (1 out of 2) Iterations: [15, 17]
Fine-tuning the circle... Iterations: [4]
Result info: 0.880241 (448080.1224, 531070.8977) 5455.6313
Finished plotting Hartlepool.
==== Middlesbrough (2 -> 216) ====
[0, 207, 216] (0, 207)
Analyzing best overall circle...  Iterations: [12, 14]
Analyzing best part circle... (1 out of 2) Iterations: [12, 14]
Fine-tuning the circle... Iterations: [7]
Result info: 0.843015 (450166.2630, 516783.7424) 4141.6191
Finished plotting Middlesbrough.
==== Redcar and Cleveland (3 -> 378) ====
[0, 364, 374, 378] (0, 364)
Analyzing best overall circle...  Iterations: [14, 22]
Analyzing best part circle... (1 out of 3) Iterations: [15, 23]
Fine-tuning the circle... Iterations: [2]
Result info: 0.762413 (462055.5827, 518365.2487) 8831.5889
Finished plotting Redcar and Cleveland.
==== Stockton-on-Tees (2 -> 671) ====
[0, 665, 671] (0

Finished plotting Slough.
==== Plymouth (4 -> 643) ====
[0, 611, 625, 639, 643] (0, 611)
Analyzing best overall circle...  Iterations: [11, 13]
Analyzing best part circle... (1 out of 4) Iterations: [9, 12]
Fine-tuning the circle... Iterations: [3]
Result info: 0.849031 (249550.8467, 56616.8503) 5042.3585
Finished plotting Plymouth.
==== Poole (5 -> 396) ====
[0, 366, 376, 384, 392, 396] (0, 366)
Analyzing best overall circle...  Iterations: [11, 16]
Analyzing best part circle... (1 out of 5) Iterations: [11, 16]
Fine-tuning the circle... Iterations: [4]
Result info: 0.801728 (402776.0547, 94003.4297) 4540.5095
Finished plotting Poole.
==== Swindon (1 -> 589) ====
[0, 589] (0, 589)
Analyzing best overall circle...  Iterations: [14, 16]
Fine-tuning the circle... Iterations: [5]
Result info: 0.834608 (418169.2951, 185314.8160) 8558.4213
Finished plotting Swindon.
==== Peterborough (1 -> 435) ====
[0, 435] (0, 435)
Analyzing best overall circle...  Iterations: [12, 17]
Fine-tuning the cir

Analyzing best overall circle...  Iterations: [14, 11]
Fine-tuning the circle... Iterations: [5]
Result info: 0.700411 (505464.3087, 232110.6740) 15095.5166
Finished plotting Central Bedfordshire.
==== Northumberland (27 -> 2875) ====
[0, 2456, 2587, 2665, 2681, 2693, 2705, 2721, 2739, 2749, 2759, 2768, 2778, 2786, 2799, 2806, 2813, 2819, 2825, 2830, 2835, 2841, 2847, 2857, 2861, 2866, 2871, 2875] (0, 2456)
Analyzing best overall circle...  Iterations: [14, 13]
Analyzing best part circle... (1 out of 27) Iterations: [5, 8]
Fine-tuning the circle... Iterations: [4]
Result info: 0.806323 (396105.9059, 590367.3020) 40005.4535
Finished plotting Northumberland.
==== Aylesbury Vale (1 -> 1229) ====
[0, 1229] (0, 1229)
Analyzing best overall circle...  Iterations: [12, 13]
Fine-tuning the circle... Iterations: [3]
Result info: 0.822958 (476841.3098, 221137.1363) 16955.3486
Finished plotting Aylesbury Vale.
==== South Bucks (1 -> 404) ====
[0, 404] (0, 404)
Analyzing best overall circle...  It

Analyzing best overall circle...  Iterations: [19, 17]
Fine-tuning the circle... Iterations: [8]
Result info: 0.677060 (428344.4371, 326635.2721) 10374.4806
Finished plotting South Derbyshire.
==== Southwark (1 -> 172) ====
[0, 172] (0, 172)
Analyzing best overall circle...  Iterations: [9, 16]
Fine-tuning the circle... Iterations: [4]
Result info: 0.694237 (533918.7068, 176852.0609) 3030.3039
Finished plotting Southwark.
==== East Devon (1 -> 1108) ====
[0, 1108] (0, 1108)
Analyzing best overall circle...  Iterations: [7, 23]
Fine-tuning the circle... Iterations: [2]
Result info: 0.774136 (316175.1074, 96403.5349) 16099.0278
Finished plotting East Devon.
==== Teignbridge (1 -> 1005) ====
[0, 1005] (0, 1005)
Analyzing best overall circle...  Iterations: [7, 10]
Fine-tuning the circle... Iterations: [3]
Result info: 0.917909 (282982.0431, 80052.4044) 14640.9120
Finished plotting Teignbridge.
==== Torridge (4 -> 3124) ====
[0, 2912, 3115, 3120, 3124] (0, 2912)
Analyzing best overall circ

Analyzing best part circle... (1 out of 6) Iterations: [17, 8]
Fine-tuning the circle... Iterations: [3]
Result info: 0.624250 (591355.1675, 191753.6240) 7344.3183
Finished plotting Rochford.
==== Tendring (12 -> 1439) ====
[0, 1139, 1222, 1288, 1321, 1338, 1347, 1369, 1402, 1408, 1417, 1433, 1439] (0, 1139)
Analyzing best overall circle...  Iterations: [5, 16]
Analyzing best part circle... (1 out of 12) Iterations: [5, 16]
Fine-tuning the circle... Iterations: [2]
Result info: 0.882325 (614425.1311, 223738.5361) 10365.2804
Finished plotting Tendring.
==== Uttlesford (1 -> 784) ====
[0, 784] (0, 784)
Analyzing best overall circle...  Iterations: [10, 19]
Fine-tuning the circle... Iterations: [3]
Result info: 0.830287 (555951.9938, 229210.4031) 14280.8066
Finished plotting Uttlesford.
==== Cheltenham (1 -> 196) ====
[0, 196] (0, 196)
Analyzing best overall circle...  Iterations: [21, 10]
Fine-tuning the circle... Iterations: [4]
Result info: 0.858685 (394697.4744, 222064.2182) 3849.8942

...

Finished plotting Birmingham.
==== Dudley (1 -> 435) ====
[0, 435] (0, 435)
Analyzing best overall circle...  Iterations: [11, 14]
Fine-tuning the circle... Iterations: [7]
Result info: 0.767490 (392829.5814, 286785.8464) 5582.3676
Finished plotting Dudley.
==== Sandwell (1 -> 297) ====
[0, 297] (0, 297)
Analyzing best overall circle...  Iterations: [11, 10]
Fine-tuning the circle... Iterations: [2]
Result info: 0.843818 (399782.9280, 291074.6666) 5217.8579
Finished plotting Sandwell.
==== Richmond upon Thames (4 -> 280) ====
[0, 262, 270, 276, 280] (0, 262)
Analyzing best overall circle...  Iterations: [7, 18]
Analyzing best part circle... (1 out of 4) Iterations: [7, 18]
Fine-tuning the circle... Iterations: [4]
Result info: 0.748655 (517020.1668, 172496.9553) 4273.5504
Finished plotting Richmond upon Thames.
==== Solihull (1 -> 520) ====
[0, 520] (0, 520)
Analyzing best overall circle...  Iterations: [11, 13]
Fine-tuning the circle... Iterations: [4]
Result info: 0.813382 (419117.15

Analyzing best overall circle...  Iterations: [16, 14]
Analyzing best part circle... (1 out of 1575) Iterations: [10, 9]
Fine-tuning the circle... Iterations: [4]
Result info: 0.587259 (129205.0846, 928497.1116) 31397.4139
Finished plotting Na h-Eileanan Siar.
==== Falkirk (1 -> 753) ====
[0, 753] (0, 753)
Analyzing best overall circle...  Iterations: [14, 11]
Fine-tuning the circle... Iterations: [3]
Result info: 0.813399 (287605.4451, 679558.3695) 9730.1808
Finished plotting Falkirk.
==== Fife (8 -> 1898) ====
[0, 1741, 1796, 1833, 1857, 1876, 1889, 1894, 1898] (0, 1741)
Analyzing best overall circle...  Iterations: [18, 22]
Analyzing best part circle... (1 out of 8) Iterations: [16, 21]
Fine-tuning the circle... Iterations: [5]
Result info: 0.679039 (337686.4816, 708322.6375) 20533.2622
Finished plotting Fife.
==== Highland (815 -> 41614) ====
[0, 24887, 30120, 30917, 31455, 31687, 31864, 32216, 32410, 32856, 33047, 33266, 33428, 33492, 33611, 33746, 33878, 33981, 34058, 34123, 3426

Finished plotting North Ayrshire.
==== Orkney Islands (240 -> 8924) ====
[0, 2660, 3410, 4052, 4730, 5030, 5480, 5706, 6009, 6134, 6291, 6478, 6605, 6675, 6727, 6773, 6827, 6887, 6967, 6991, 7061, 7113, 7142, 7166, 7211, 7248, 7286, 7307, 7346, 7383, 7399, 7458, 7476, 7496, 7543, 7557, 7572, 7594, 7609, 7622, 7633, 7646, 7667, 7689, 7697, 7709, 7720, 7741, 7762, 7779, 7791, 7809, 7823, 7831, 7844, 7857, 7866, 7874, 7885, 7895, 7907, 7913, 7931, 7940, 7951, 7959, 7971, 7988, 8003, 8016, 8023, 8040, 8050, 8057, 8073, 8078, 8090, 8095, 8105, 8115, 8121, 8128, 8134, 8139, 8152, 8160, 8167, 8173, 8179, 8185, 8192, 8200, 8206, 8211, 8222, 8230, 8241, 8249, 8255, 8261, 8270, 8278, 8283, 8290, 8297, 8303, 8310, 8315, 8322, 8327, 8332, 8340, 8345, 8351, 8358, 8364, 8369, 8374, 8381, 8389, 8396, 8402, 8407, 8414, 8420, 8425, 8430, 8435, 8440, 8445, 8451, 8455, 8460, 8467, 8472, 8476, 8480, 8485, 8490, 8495, 8500, 8504, 8509, 8513, 8518, 8523, 8528, 8534, 8539, 8543, 8549, 8553, 8557, 8562, 8566,

Analyzing best overall circle...  Iterations: [17, 20]
Fine-tuning the circle... Iterations: [6]
Result info: 0.800365 (259362.5791, 708140.7675) 26786.3596
Finished plotting Stirling.
==== Aberdeen City (12 -> 700) ====
[0, 654, 658, 663, 668, 672, 676, 680, 684, 688, 692, 696, 700] (0, 654)
Analyzing best overall circle...  Iterations: [8, 18]
Analyzing best part circle... (1 out of 12) Iterations: [8, 18]
Fine-tuning the circle... Iterations: [3]
Result info: 0.786833 (389541.8013, 808130.2029) 7684.9705
Finished plotting Aberdeen City.
==== Aberdeenshire (98 -> 5169) ====
[0, 4714, 4732, 4745, 4750, 4755, 4762, 4770, 4776, 4782, 4787, 4791, 4796, 4801, 4807, 4813, 4818, 4823, 4829, 4833, 4838, 4844, 4849, 4853, 4857, 4862, 4868, 4873, 4879, 4883, 4888, 4893, 4897, 4901, 4906, 4910, 4915, 4919, 4923, 4928, 4933, 4937, 4941, 4945, 4949, 4953, 4957, 4961, 4966, 4971, 4975, 4979, 4983, 4988, 4992, 4996, 5000, 5004, 5008, 5012, 5016, 5021, 5025, 5029, 5033, 5037, 5041, 5045, 5049, 5053,

Finished plotting West Lothian.
==== Angus (2 -> 1422) ====
[0, 1416, 1422] (0, 1416)
Analyzing best overall circle...  Iterations: [4, 11]
Analyzing best part circle... (1 out of 2) Iterations: [4, 11]
Fine-tuning the circle... Iterations: [3]
Result info: 0.891121 (344547.1714, 760004.9095) 26366.7120
Finished plotting Angus.
==== Dundee City (1 -> 187) ====
[0, 187] (0, 187)
Analyzing best overall circle...  Iterations: [15, 15]
Fine-tuning the circle... Iterations: [3]
Result info: 0.661115 (339242.4082, 732094.9215) 4364.6079
Finished plotting Dundee City.
==== North Lanarkshire (1 -> 740) ====
[0, 740] (0, 740)
Analyzing best overall circle...  Iterations: [12, 15]
Fine-tuning the circle... Iterations: [2]
Result info: 0.793356 (279877.6800, 665002.0119) 12265.5070
Finished plotting North Lanarkshire.
==== Glasgow City (2 -> 643) ====
[0, 399, 643] (0, 399)
Analyzing best overall circle...  Iterations: [8, 14]
Analyzing best part circle... (1 out of 2) Iterations: [10, 13]
Fine-t